## Impact of Pollutants on AQI

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xlsxwriter

In [5]:
DATA_PATH = "BIDM_Dataset.csv"
OUTPUT_DIR = "aqi_analysis_outputs"
EXCEL_PATH = os.path.join(OUTPUT_DIR, "aqi_analysis.xlsx")
POLLUTANTS = ['co','no','no2','o3','so2','pm2_5','pm10','nh3']
TARGET = 'aqi'
MAX_SAMPLES = 20000
RANDOM_STATE = 42
N_TREES = 200
N_PERM = 12
PDP_GRID_POINTS = 50

In [7]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [9]:
# 1) Load dataset
df = pd.read_csv(DATA_PATH)

# 2) Check required columns
missing_cols = [c for c in POLLUTANTS + [TARGET] if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")

# 3) Subset & drop NA
df_sub = df[POLLUTANTS + [TARGET]].dropna()
if df_sub.shape[0] == 0:
    raise ValueError("No rows left after dropping NA. Check dataset.")

# 4) Sampling for speed
if df_sub.shape[0] > MAX_SAMPLES:
    df_sub = df_sub.sample(n=MAX_SAMPLES, random_state=RANDOM_STATE)

X = df_sub[POLLUTANTS].astype(float)
y = df_sub[TARGET].astype(float)

# 5) Train/test split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

# 6) Fit Random Forest
rf = RandomForestRegressor(n_estimators=N_TREES, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_tr, y_tr)

# 7) Performance metrics
def rmse(a,b): return np.sqrt(mean_squared_error(a,b))
perf = {
    'train_r2': float(r2_score(y_tr, rf.predict(X_tr))),
    'test_r2': float(r2_score(y_te, rf.predict(X_te))),
    'train_rmse': float(rmse(y_tr, rf.predict(X_tr))),
    'test_rmse': float(rmse(y_te, rf.predict(X_te))),
    'train_mae': float(mean_absolute_error(y_tr, rf.predict(X_tr))),
    'test_mae': float(mean_absolute_error(y_te, rf.predict(X_te)))
}
perf_df = pd.DataFrame([perf])

# 8) Permutation importance
perm = permutation_importance(rf, X_te, y_te, n_repeats=N_PERM, random_state=RANDOM_STATE, n_jobs=-1)
perm_df = pd.DataFrame({
    'feature': POLLUTANTS,
    'perm_mean': perm.importances_mean,
    'perm_std': perm.importances_std
}).sort_values('perm_mean', ascending=False).reset_index(drop=True)

# 9) Mutual information
mi_scores = mutual_info_regression(X_tr.fillna(0), y_tr, random_state=RANDOM_STATE)
mi_df = pd.DataFrame({'feature': POLLUTANTS, 'mi_score': mi_scores}).sort_values('mi_score', ascending=False).reset_index(drop=True)

# 10) RF built-in importances
rf_builtin_df = pd.DataFrame({'feature': POLLUTANTS, 'rf_builtin_imp': rf.feature_importances_}).sort_values('rf_builtin_imp', ascending=False).reset_index(drop=True)

# 11) Combined ranking
combined_df = perm_df.merge(mi_df, on='feature').merge(rf_builtin_df, on='feature')

# 12) PDP computations for top features (top 3 by permutation importance)
top_feats = perm_df['feature'].head(3).tolist()
pdp_tables = []
for feat in top_feats:
    series = X[feat]
    grid = np.linspace(np.percentile(series, 1), np.percentile(series, 99), PDP_GRID_POINTS)
    # use a baseline sample from X_te
    baseline = X_te.copy()
    if baseline.shape[0] > 2000:
        baseline = baseline.sample(n=2000, random_state=RANDOM_STATE)
    means = []
    for val in grid:
        tmp = baseline.copy()
        tmp[feat] = val
        preds = rf.predict(tmp)
        means.append(preds.mean())
    pdp_df = pd.DataFrame({'feature_value': grid, 'predicted_aqi': means})
    pdp_tables.append((feat, pdp_df))
    # save PNG
    plt.figure(figsize=(6,3.5))
    plt.plot(grid, means, marker='o', linewidth=1)
    plt.xlabel(feat)
    plt.ylabel('Predicted AQI (PDP)')
    plt.title(f'PDP: {feat} → AQI')
    plt.grid(True)
    png = os.path.join(OUTPUT_DIR, f"pdp_{feat}_aqi.png")
    plt.tight_layout()
    plt.savefig(png, dpi=150)
    plt.close()

# 13) Export all results to one Excel with multiple sheets
with pd.ExcelWriter(EXCEL_PATH, engine='xlsxwriter') as writer:
    perf_df.to_excel(writer, sheet_name='performance', index=False)
    perm_df.to_excel(writer, sheet_name='permutation_importance', index=False)
    mi_df.to_excel(writer, sheet_name='mutual_information', index=False)
    rf_builtin_df.to_excel(writer, sheet_name='rf_builtin_importance', index=False)
    combined_df.to_excel(writer, sheet_name='combined_ranking', index=False)
    # PDP sheets
    for feat, ptab in pdp_tables:
        sheetname = f'pdp_{feat}'
        # Excel sheet names max length ~31 chars
        ptab.to_excel(writer, sheet_name=sheetname[:31], index=False)
    # Save a sample of raw training data (first 100 rows) for reference in Power BI if needed
    pd.concat([X_tr.head(100).reset_index(drop=True), pd.DataFrame({'aqi': y_tr.reset_index(drop=True).iloc[:100]})], axis=1).to_excel(writer, sheet_name='sample_train', index=False)

print("AQI analysis Excel saved to:", EXCEL_PATH)
print("PDP PNG(s) saved in folder:", OUTPUT_DIR)

AQI analysis Excel saved to: aqi_analysis_outputs/aqi_analysis.xlsx
PDP PNG(s) saved in folder: aqi_analysis_outputs


### Impact of Pollutants on Temperature

In [14]:
DATA_PATH = "BIDM_Dataset.csv"
OUTPUT_DIR = "temp_analysis_outputs"
EXCEL_PATH = os.path.join(OUTPUT_DIR, "temp_analysis.xlsx")
POLLUTANTS = ['co','no','no2','o3','so2','pm2_5','pm10','nh3']
MAX_SAMPLES = 20000
RANDOM_STATE = 42
N_TREES = 200
N_PERM = 12
PDP_GRID_POINTS = 50

In [16]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [18]:
# Load data
df = pd.read_csv(DATA_PATH)

# Detect temperature-like column
temp_candidates = [c for c in df.columns if 'temp' in c.lower() or 'atemp' in c.lower() or 'temperature' in c.lower()]
if not temp_candidates:
    raise ValueError("No temperature-like column found. Ensure column contains substring 'temp' or 'temperature' or 'atemp'.")
TEMP_COL = temp_candidates[0]
print("Using temperature column:", TEMP_COL)

# Check pollutant columns exist
missing_cols = [c for c in POLLUTANTS + [TEMP_COL] if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")

# Subset and drop NA
df_sub = df[POLLUTANTS + [TEMP_COL]].dropna()
if df_sub.shape[0] == 0:
    raise ValueError("No data after dropping NA — check dataset.")

# Sampling
if df_sub.shape[0] > MAX_SAMPLES:
    df_sub = df_sub.sample(n=MAX_SAMPLES, random_state=RANDOM_STATE)

X = df_sub[POLLUTANTS].astype(float)
y = df_sub[TEMP_COL].astype(float)

# Train/test split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

# Fit RF
rf = RandomForestRegressor(n_estimators=N_TREES, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_tr, y_tr)

# Performance
def rmse(a,b): return np.sqrt(mean_squared_error(a,b))
perf = {
    'train_r2': float(r2_score(y_tr, rf.predict(X_tr))),
    'test_r2': float(r2_score(y_te, rf.predict(X_te))),
    'train_rmse': float(rmse(y_tr, rf.predict(X_tr))),
    'test_rmse': float(rmse(y_te, rf.predict(X_te))),
    'train_mae': float(mean_absolute_error(y_tr, rf.predict(X_tr))),
    'test_mae': float(mean_absolute_error(y_te, rf.predict(X_te)))
}
perf_df = pd.DataFrame([perf])

# Permutation importance
perm = permutation_importance(rf, X_te, y_te, n_repeats=N_PERM, random_state=RANDOM_STATE, n_jobs=-1)
perm_df = pd.DataFrame({
    'feature': POLLUTANTS,
    'perm_mean': perm.importances_mean,
    'perm_std': perm.importances_std
}).sort_values('perm_mean', ascending=False).reset_index(drop=True)

# Mutual information
mi_scores = mutual_info_regression(X_tr.fillna(0), y_tr, random_state=RANDOM_STATE)
mi_df = pd.DataFrame({'feature': POLLUTANTS, 'mi_score': mi_scores}).sort_values('mi_score', ascending=False).reset_index(drop=True)

# RF builtin importances
rf_builtin_df = pd.DataFrame({'feature': POLLUTANTS, 'rf_builtin_imp': rf.feature_importances_}).sort_values('rf_builtin_imp', ascending=False).reset_index(drop=True)

# Combined
combined_df = perm_df.merge(mi_df, on='feature').merge(rf_builtin_df, on='feature')

# PDPs top 3
top_feats = perm_df['feature'].head(3).tolist()
pdp_tables = []
for feat in top_feats:
    series = X[feat]
    grid = np.linspace(np.percentile(series, 1), np.percentile(series, 99), PDP_GRID_POINTS)
    baseline = X_te.copy()
    if baseline.shape[0] > 2000:
        baseline = baseline.sample(n=2000, random_state=RANDOM_STATE)
    means = []
    for val in grid:
        tmp = baseline.copy()
        tmp[feat] = val
        preds = rf.predict(tmp)
        means.append(preds.mean())
    pdp_df = pd.DataFrame({'feature_value': grid, f'predicted_{TEMP_COL}': means})
    pdp_tables.append((feat, pdp_df))
    # save PNG
    plt.figure(figsize=(6,3.5))
    plt.plot(grid, means, marker='o', linewidth=1)
    plt.xlabel(feat)
    plt.ylabel(f'Predicted {TEMP_COL} (PDP)')
    plt.title(f'PDP: {feat} → {TEMP_COL}')
    plt.grid(True)
    png = os.path.join(OUTPUT_DIR, f"pdp_{feat}_temp_{TEMP_COL}.png")
    plt.tight_layout()
    plt.savefig(png, dpi=150)
    plt.close()

# Export to Excel
with pd.ExcelWriter(EXCEL_PATH, engine='xlsxwriter') as writer:
    perf_df.to_excel(writer, sheet_name='performance', index=False)
    perm_df.to_excel(writer, sheet_name='permutation_importance', index=False)
    mi_df.to_excel(writer, sheet_name='mutual_information', index=False)
    rf_builtin_df.to_excel(writer, sheet_name='rf_builtin_importance', index=False)
    combined_df.to_excel(writer, sheet_name='combined_ranking', index=False)
    for feat, ptab in pdp_tables:
        sheetname = f'pdp_{feat}'
        ptab.to_excel(writer, sheet_name=sheetname[:31], index=False)
    pd.concat([X_tr.head(100).reset_index(drop=True), pd.DataFrame({TEMP_COL: y_tr.reset_index(drop=True).iloc[:100]})], axis=1).to_excel(writer, sheet_name='sample_train', index=False)

print("Temperature analysis Excel saved to:", EXCEL_PATH)
print("PDP PNG(s) saved in folder:", OUTPUT_DIR)

Using temperature column: temp_avg
Temperature analysis Excel saved to: temp_analysis_outputs/temp_analysis.xlsx
PDP PNG(s) saved in folder: temp_analysis_outputs
